In [1]:
pip install fastapi uvicorn httpx
# Install FastAPI framework for building APIs
# Install Uvicorn ASGI server to run the FastAPI app
# Install HTTPX for making asynchronous HTTP requests




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00


In [4]:
 # Importing necessary modules
from fastapi import FastAPI, HTTPException         # FastAPI for building the API, HTTPException for error handling
import httpx, time                                 # httpx for async HTTP requests, time for measuring request duration
from collections import deque                      # deque used for a fixed-size window of values
from typing import List                            # Type hinting

# Initialize FastAPI app
app = FastAPI()

# Set the maximum size of the window
WINDOW_SIZE = 10

# Create separate deques (fixed-size queues) for each type of number
# Each deque will store the most recent unique numbers up to WINDOW_SIZE
number_queues = {
    'p': deque(maxlen=WINDOW_SIZE),  # Prime numbers
    'f': deque(maxlen=WINDOW_SIZE),  # Fibonacci numbers
    'e': deque(maxlen=WINDOW_SIZE),  # Even numbers
    'r': deque(maxlen=WINDOW_SIZE)   # Random numbers
}

# Define the external API endpoints for each number type
ENDPOINTS = {
    'p': "http://20.244.56.144/evaluation-service/primes",
    'f': "http://20.244.56.144/evaluation-service/fibo",
    'e': "http://20.244.56.144/evaluation-service/even",
    'r': "http://20.244.56.144/evaluation-service/rand"
}

# API route to fetch numbers from the test server
@app.get("/numbers/{number_id}")
async def fetch_numbers(number_id: str):
    # Check if number_id is valid (must be one of: p, f, e, r)
    if number_id not in number_queues:
        raise HTTPException(status_code=400, detail="Invalid number ID")

    # Copy the current state of the window before fetching new numbers
    prev_window = list(number_queues[number_id])
    numbers_fetched = []  # To store numbers received from the API

    try:
        # Record the start time to measure if the response takes more than 0.5 seconds
        start_time = time.time()

        # Asynchronous HTTP client with timeout of 0.5 seconds
        async with httpx.AsyncClient(timeout=0.5) as client:
            res = await client.get(ENDPOINTS[number_id])  # Make GET request to the appropriate API

        # If the request failed or took too long, raise an exception
        if res.status_code != 200 or (time.time() - start_time) > 0.5:
            raise Exception("Timeout or Error")

        # Parse JSON response and extract the "numbers" list
        data = res.json()
        numbers_fetched = data.get("numbers", [])

    except Exception:
        # If there's an error or timeout, return the previous state and no new numbers
        return {
            "windowPrevState": prev_window,     #huh bkihiu hio juh
            "windowCurrState": prev_window,         #jy yjfh jg ujh jgh
            "numbers": [],                    #khy hi ofjyg9 epjyu
            "avg": round(sum(prev_window) / len(prev_window), 2) if prev_window else 0.0 #lehfojey uftye ofkg
        }

    # Append only unique numbers to the deque (ignore duplicates)
    for num in numbers_fetched:
    # Append only unique numbers to the deque (ignore duplicates)
        if num not in number_queues[number_id]:
    # Append only unique numbers to the deque (ignore duplicates)
            number_queues[number_id].append(num)
    # Append only unique numbers to the deque (ignore duplicates)

    # Current window after adding new numbers
    curr_window = list(number_queues[number_id])

    # Calculate average of current window
    average = round(sum(curr_window) / len(curr_window), 2) if curr_window else 0.0

    # Return the previous state, current state, new numbers fetched, and the average
    return {
        "windowPrevState": prev_window, # Return the previous state, current state, new numbers fetched, and the average
        "windowCurrState": curr_window,    # Return the previous state, current state, new numbers fetched, and the average
        "numbers": numbers_fetched,       # Return the previous state, current state, new numbers fetched, and the average
        "avg": average
    }
